In [ ]:
from datetime import datetime
import csv
import os
TRADE_LOG_FILE = "trade_log.csv"


def check_trades(symbol, file_path=TRADE_LOG_FILE):
    if not os.path.exists(file_path):
        return []

    # Read trades for the given symbol from the CSV file
    trades = []
    with open(file_path, mode="r") as file:
        reader = csv.DictReader(file)
        today = datetime.now().strftime("%Y-%m-%d")
        for row in reader:
            # Check for trade date 
            trade_date = datetime.strptime(row['timestamp'],'%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")

            # Check for the symbol and if it is traded today and the status is successful
            if row["symbol"] == symbol and trade_date == today and row['status'] == 'success':
                trades.append(row)
    return trades


# If trades for any symbol are equal to two or if its recently traded like 10 mins ago then don't trade again on that symbol 

from datetime import datetime
import csv

def can_trade(symbol, file_path=TRADE_LOG_FILE):
    trades = check_trades(symbol, file_path)

    # Rule 1: Block if already 2 or more trades
    if len(trades) >= 2:
        return False

    # Rule 2: Find the most recent trade for this symbol
    latest_trade_time = None
    with open(file_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["symbol"] == symbol and row["status"] == "success":
                # Parse timestamp with flexible format
                try:
                    timestamp = datetime.strptime(row['timestamp'], '%d-%m-%Y %H:%M')
                except ValueError:
                    timestamp = datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')

                if latest_trade_time is None or timestamp > latest_trade_time:
                    latest_trade_time = timestamp

    # Rule 3: If a trade exists and it's within 10 minutes, block
    if latest_trade_time and (datetime.now() - latest_trade_time).total_seconds() < 600:
        return False

    # Otherwise, allow trade
    return True



can_trade(symbol='NSE:RITES-EQ', file_path=TRADE_LOG_FILE)


False

In [ ]:
no_of_stocks = 3

def can_trade(symbol, file_path=TRADE_LOG_FILE, stock_count=no_of_stocks):

    trades = check_trades(symbol, file_path)

    if stock_count <3:
        if len(trades) >= 3:
            return False

    # Rule 1: Block if already 2 or more trades
    if stock_count>=3:
        if len(trades) >= 2:
            return False

    # Rule 2: Find the most recent trade for this symbol
    latest_trade_time = None
    with open(file_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["symbol"] == symbol and row["status"] == "success":
                # Parse timestamp with flexible format
                try:
                    timestamp = datetime.strptime(row['timestamp'], '%d-%m-%Y %H:%M')
                except ValueError:
                    timestamp = datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')

                if latest_trade_time is None or timestamp > latest_trade_time:
                    latest_trade_time = timestamp

    # Rule 3: If a trade exists and it's within 10 minutes, block
    if latest_trade_time and (datetime.now() - latest_trade_time).total_seconds() < 600:
        return False

    # Otherwise, allow trade
    return True


NameError: name 'TRADE_LOG_FILE' is not defined

In [5]:
filtered_stocks = {'RELIANCE',"JIO","ZOMATO","HUL"}

for i in filtered_stocks:
    print(i)
    a = 4
    b = 6

    def add_stocks(a,b,count = len(filtered_stocks)):
        return i, a * b + count
    
    print(add_stocks(a, b))



JIO
('JIO', 28)
HUL
('HUL', 28)
RELIANCE
('RELIANCE', 28)
ZOMATO
('ZOMATO', 28)


In [64]:
from src.tradingsetup.login.authentication import auto_login
from dotenv import load_dotenv
from src.tradingsetup.login.auth import get_fyers_instance, is_access_token_valid
from src.tradingsetup.utlis.logger import log

log("Starting trading script...")


# Step 1: Authentication
log("Authenticating with Fyers to use its API...")
try:
    if not is_access_token_valid():
        auto_login()
        load_dotenv(override=True)
    log("Authentication is already done as access token is valid.")
    log("Sleeping for 3 sec to make sure ACCESS_TOKEN is properly fetched and picked up for WebSocket")
    time.sleep(3)
except Exception as e:
    log(f"Auth failed: {e}")
    exit(1)

# This uses the valid access_token from .env 
fyers = get_fyers_instance()
fyers.get_profile()


2025-10-12 17:29:35,739 [INFO] Starting trading script...
2025-10-12 17:29:35,741 [INFO] Authenticating with Fyers to use its API...
2025-10-12 17:29:39,085 [INFO] Access token is not valid.
2025-10-12 17:29:41,026 [INFO] verify_client_id success
2025-10-12 17:29:41,030 [INFO] generate_totp success
2025-10-12 17:29:41,635 [INFO] verify_totp_result success
2025-10-12 17:29:42,937 [INFO] verify_pin_result success
2025-10-12 17:29:43,554 [INFO] token_result success
2025-10-12 17:29:44,148 [INFO] validate_authcode success
2025-10-12 17:29:44,157 [INFO] Authentication successful
2025-10-12 17:29:44,185 [INFO] Authentication is already done as access token is valid.
2025-10-12 17:29:44,185 [INFO] Sleeping for 3 sec to make sure ACCESS_TOKEN is properly fetched and picked up for WebSocket
2025-10-12 17:29:44,185 [INFO] Auth failed: type object 'datetime.time' has no attribute 'sleep'
2025-10-12 17:29:44,185 [INFO] Creating FyersModel instance with access token...
2025-10-12 17:29:44,185 [INFO

{'s': 'error',
 'code': -8,
 'message': 'Your token has expired. Please generate a token'}

In [120]:
from fyers_apiv3 import fyersModel

access_token='eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbzdHT3Frb19Sc203QXVRMldQeUdfUXl0Q0pQWHVtd2lTTWltWFpFc1VHM0hJa01JSzJscGQtN2FZS2hMVDZONnNjWklLakgwSVAtQTF0cFNvR0tCN1ljYzlrWmYwNjBWaVp4dXRaelRDTEJFUGlwZz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiI0NmQwNTgwMjdjZjc4NDhjMDRiYWE5YzY4ZmM4MjZiN2Q4ZGU1MmY4MzRjZTA5MDNmZGUxY2Y3MyIsImlzRGRwaUVuYWJsZWQiOiJZIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWFIwNDc1MCIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzYwNDAxODAwLCJpYXQiOjE3NjAzMjI0NzQsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc2MDMyMjQ3NCwic3ViIjoiYWNjZXNzX3Rva2VuIn0.I66vV574tjnRy2deg3Td_rJGoyhDInX6k-ZI94VXy9k'

fyers = fyersModel.FyersModel(token=access_token,is_async=False,client_id=client_id,log_path="")



## After this point you can call the relevant apis and get started with

####################################################################################################################
"""
1. User Apis : This includes (Profile,Funds,Holdings)
"""

print(f'Customer Profile: \n {fyers.get_profile()['data']}')  ## This will provide us with the user related data 
print('\n')
print(f'Customer Account information: \n {fyers.funds()['fund_limit']}')        ## This will provide us with the funds the user has 
print('\n')
print(f'Customer Investment details: \n {fyers.holdings()['overall']}')    ## This will provide the available holdings the user has 
print('\n')
print("Fetching and filtering stocks...")




Customer Profile: 
 {'fy_id': 'XR04750', 'name': 'REDEKAR SHIVDATTA SURESH', 'image': None, 'display_name': None, 'pin_change_date': '06-01-2022 09:51:24', 'email_id': 'sxxxxxxxxxxxxxxr@gmail.com', 'pwd_change_date': '06-01-2022 09:50:31', 'PAN': 'BMxxxxxx1N', 'mobile_number': '98xxxxxx43', 'totp': True, 'pwd_to_expire': -1226, 'ddpi_enabled': True, 'mtf_enabled': False}


Customer Account information: 
 [{'id': 1, 'title': 'Total Balance', 'equityAmount': 35017.119999999995, 'commodityAmount': 0}, {'id': 2, 'title': 'Utilized Amount', 'equityAmount': 219.98956594652032, 'commodityAmount': 0}, {'id': 3, 'title': 'Clear Balance', 'equityAmount': 34999.28043405348, 'commodityAmount': 0}, {'id': 4, 'title': 'Realized Profit and Loss', 'equityAmount': -202.1500000000003, 'commodityAmount': 0}, {'id': 5, 'title': 'Collaterals', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 6, 'title': 'Fund Transfer', 'equityAmount': 0, 'commodityAmount': 0}, {'id': 7, 'title': 'Receivables', 'equityAmou

In [121]:
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'XR04750',
  'name': 'REDEKAR SHIVDATTA SURESH',
  'image': None,
  'display_name': None,
  'pin_change_date': '06-01-2022 09:51:24',
  'email_id': 'sxxxxxxxxxxxxxxr@gmail.com',
  'pwd_change_date': '06-01-2022 09:50:31',
  'PAN': 'BMxxxxxx1N',
  'mobile_number': '98xxxxxx43',
  'totp': True,
  'pwd_to_expire': -1226,
  'ddpi_enabled': True,
  'mtf_enabled': False}}

In [ ]:
# for one stock statergy

from datetime import datetime, time


class SingleStockStrategy:
    def __init__(self, symbol, RR, fyers):
        self.symbol = symbol 
        self.RR = RR
        self.candles = self._get_candles(symbol)
        self.fyers = fyers

    def _get_candles(self, symbol):
        log(f"Fetching candles for {symbol}...")
        
        data = fyers.history({
                    "symbol": symbol,
                    "resolution": "5",
                    "date_format": "1",
                    "range_from": datetime.now().date(),
                    "range_to": datetime.now().date(),
                    "cont_flag": "1"
                })
        candle = data.get("candles", [])
            
        # Optional: Convert timestamp to readable format for inspection
        candles = []

        for c in candle:
            dt = datetime.fromtimestamp(c[0])
            if dt.time() > time(9, 55):
                candles.append({
                    "daytime": datetime.fromtimestamp(c[0]).strftime('%Y-%m-%d %H:%M'),
                    "day_time": dt.strftime("%H:%M"),
                    "open": c[1],
                    "high": c[2],
                    "low": c[3],
                    "close": c[4],
                    "volume": c[5]
                })

        return candles

    def _get_trigger_candle(self):
            
        for candle in self.candles:
            if candle["day_time"] == "10:00":
                return candle
        return None

    def evaluate_trade_signal(self):
        trigger_candle = self._get_trigger_candle()
        
        if not trigger_candle:
            log("No 10.00 AM trigger candle found.")
            return False
        
        trigger_high = trigger_candle["high"]
        trigger_low = trigger_candle["low"]
        trigger_range = trigger_high - trigger_low

        try:
            for candle in self.candles: 
                if candle["day_time"] < "10:00":
                    continue

                # Buy conditions
                if candle["high"]> trigger_high:
                    log("Trade signal evaluated as Buy.")
                    signal = {
                        "timestamp": candle["daytime"],
                        "action": "BUY",
                        "entry_price": trigger_high,
                        "stop_loss": trigger_low,
                        "target":  round(trigger_high + self.RR * trigger_range, 0)
                    }
                    log(f"Buy Triggered for {self.symbol} SETUP at {candle['daytime']}")
                    return signal
                
                # Sell conditions
                if candle["low"] < trigger_low:
                    log("Trade signal evaluated as Sell.")
                    signal = {
                        "timestamp": candle["daytime"],
                        "action": "SELL",
                        "entry_price": trigger_low,
                        "stop_loss": trigger_high,
                        "target": round(trigger_low - self.RR * trigger_range, 0)
                    }
                    log(f"Sell Triggered for {self.symbol} SETUP at {candle['daytime']}")
                    return signal
                
        except Exception as e:
            log(f"Trade signal evaluated as invalid due to :{e}")
            return False




In [138]:
symbol = "NSE:TATAMOTORS-EQ"

signal = SingleStockStrategy(symbol, 3, fyers).evaluate_trade_signal()
signal

2025-10-13 20:56:57,410 [INFO] Fetching candles for NSE:TATAMOTORS-EQ...
2025-10-13 20:56:58,391 [INFO] Trade signal evaluated as Buy.
2025-10-13 20:56:58,393 [INFO] Buy Triggered for NSE:TATAMOTORS-EQ SETUP at 2025-10-13 10:05


{'timestamp': '2025-10-13 10:05',
 'action': 'BUY',
 'entry_price': 671,
 'stop_loss': 669.25,
 'target': 676.0}

In [139]:
from src.tradingsetup.trade_logic.ema_strategy import place_trade

place_trade(fyers, symbol, signal["entry_price"], signal["stop_loss"], signal["target"], signal["timestamp"])

2025-10-13 20:56:59,526 [INFO] Trade for NSE:TATAMOTORS-EQ at 2025-10-13 10:05 is outside the trading window. as current time is 20:56:59.526426 hence Skipping.


In [123]:
evaluate_trade_signal(candles,3)

2025-10-13 20:32:13,806 [INFO] Trade signal evaluated as Buy.
2025-10-13 20:32:13,808 [INFO] Buy Triggered for NSE:TATAMOTORS-EQ SETUP at 10:05


{'timestamp': '10:05',
 'action': 'BUY',
 'entry_price': 671,
 'stop_loss': 669.25,
 'target': 676.0}

In [116]:
candles

[]

In [12]:
from tradingsetup.utlis.logger import log

message= {'ltp': 1094.5, 'vol_traded_today': 12189, 'last_traded_time': 1759204111, 'exch_feed_time': 1759204111, 'bid_size': 23, 'ask_size': 45, 'bid_price': 1094.5, 'ask_price': 1095.2, 'last_traded_qty': 14, 'tot_buy_qty': 81094, 'tot_sell_qty': 69679, 'avg_trade_price': 1092.33, 'low_price': 1088.0, 'high_price': 1095.4, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 1088.0, 'prev_close_price': 1089.5, 'type': 'sf', 'symbol': 'NSE:AUROPHARMA-EQ', 'ch': 5.0, 'chp': 0.4589}
message1= {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
message2= {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
message3= {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}

messages = [message,message1,message2,message3]



def msg_logger(message):        
    if list(message.keys())[0]=='ltp':
        pass
    else:
        log(f"Response: {message}")

for msg in messages:       
    msg_logger(msg)

2025-10-01 19:56:34,531 [INFO] Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
2025-10-01 19:56:34,536 [INFO] Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
2025-10-01 19:56:34,539 [INFO] Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}


In [11]:

list(message.keys())[0]=='ltp'

True

In [ ]:
from datetime import datetime, timedelta, time

days = 60
end_time = datetime.now() - timedelta(days=1)

payload = {
        "symbol": "symbol",
        "resolution": "D",
        "date_format": "0",
        "range_from": str(int((datetime.now() - timedelta(days=days+1)).timestamp())),
        "range_to": str(int(end_time.timestamp())),
        "cont_flag": "1"
    }

payload

{'symbol': 'symbol',
 'resolution': 'D',
 'date_format': '0',
 'range_from': '1754059492',
 'range_to': '1759243492',
 'cont_flag': '1'}